In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle

from Environments.Stochastic_GridWorld8Actions import GridWorld

In [ ]:
def smoothing_data(rewards_across_seeds, traps_across_seeds, variance_across_seeds):

    kernel_size = 11000
    kernel = np.ones(kernel_size) / kernel_size

    all_rewards = []
    for data in rewards_across_seeds:
        rewards = np.convolve(data, kernel)
        all_rewards.append(rewards[kernel_size:-kernel_size])

    all_traps = []
    for data in traps_across_seeds:
        traps = data
        all_traps.append(traps)

    return all_rewards, all_traps#, all_variance_rewards


In [4]:
transitions_variance = []

file = open('test_transitions_for_variance_1.0.pickle', 'rb')
# dump information to that file
transition = pickle.load(file)
transitions_variance.append(transition)
# close the file
file.close()

# file = open('test_transitions_for_variance_0.3.pickle', 'rb')
# # dump information to that file
# transition = pickle.load(file)
# transitions_variance.append(transition)
# # close the file
# file.close()

# file = open('test_transitions_for_variance_0.5.pickle', 'rb')
# # dump information to that file
# transition = pickle.load(file)
# transitions_variance.append(transition)
# # close the file
# file.close()

# file = open('test_transitions_for_variance_0.7.pickle', 'rb')
# # dump information to that file
# transition = pickle.load(file)
# transitions_variance.append(transition)
# # close the file
# file.close()

In [ ]:
for beta_data in [0,0.2,0.3,0.4,0.6,0.8]:
    env_obj = GridWorld(x_dim = 7, y_dim = 7, deterministic_T_prob = 1.0, reward_location=42) 
    env_obj.state_transitions = transitions_variance[0]

    # env_obj.set_penalty(2, 1)
    # env_obj.set_penalty(4, 3)
    # env_obj.set_penalty(3, 4)

    env_obj.set_penalty(2, 1)
    env_obj.set_penalty(4, 3)
    env_obj.set_penalty(3, 4)


    traps_across_seeds = []
    rewards_across_seeds = []
    std_across_seeds = []

    trajectory_heat_map_arr = []
    q_values_across_seeds = []

    for seed in range(4):

        EPISODES = 800000
        
        traps = np.zeros(EPISODES)
        rewards_arr_episodes = []
        std_per_episode = []

        agent_obj = Table_Q_Learning(lr = 0.01, state_space=49, annealing_coefficient =  0.999999,
                                      beta = beta_data, seed=seed)
        trajectory_heat_map = np.zeros((7,7))
        q_values_arr = []
        
        for i in range(EPISODES): #episodes
        
            state = env_obj.reset()[0]
            mean_episodic_reward = 0
            #variance_episodic_reward = 0
            #std_episodic_reward = 0
            
            for j in range(1,5001): #steps    

                action, q, all_q = agent_obj.take_action( state ) 
                new_state, reward, done, _ = env_obj.step(state, action)
                new_state = new_state[0]
                
                tempx = (reward - mean_episodic_reward)
                mean_episodic_reward += tempx / j


                agent_obj.update(state, action, new_state, reward, done, env_obj)


               
                x, y = np.where(env_obj.grid == state)
                trajectory_heat_map[x[0]][y[0]] +=1
            
                
                if(done == 1):
                    
                    x, y = np.where(env_obj.grid == new_state)
                    trajectory_heat_map[x[0]][y[0]] +=1

                     
                    # if i % 1000 == 0 and i > 140000:
                    #     print("Episode ",i, " Epsilon ",agent_obj.epsilon, " mean reward ",mean_episodic_reward)
                    if(env_obj.state_attributes[new_state] == 2):
                        traps[i] = 1
                    break

                state = new_state
        

            rewards_arr_episodes.append(mean_episodic_reward)
            #q_values_arr.append(agent_obj.q_table)
            #std_per_episode.append(std_episodic_reward)

        traps_across_seeds.append(traps)
        rewards_across_seeds.append(rewards_arr_episodes)
        std_across_seeds.append(std_per_episode)
        trajectory_heat_map_arr.append(trajectory_heat_map)
        q_values_across_seeds.append(agent_obj.q_table)

    all_rewards, all_traps = smoothing_data(rewards_across_seeds, traps_across_seeds, std_across_seeds)
    # with open('Prospect_framework_reward_zeta_test_'+str(zeta_data)+'.pickle', 'wb') as file:
    #     pickle.dump(all_rewards, file)

    with open('Prospect_framework_traps_zeta_test_'+str(beta_data)+'.pickle', 'wb') as file:
        pickle.dump(all_traps, file)

    # # with open('Prospect_framework_variance_'+determinism+'.pickle', 'wb') as file:
    # #     pickle.dump(all_variance_rewards, file)

    with open('Prospect_framework_heatmap_beta_test_'+str(beta_data)+'.pickle', 'wb') as file:
        pickle.dump(trajectory_heat_map_arr, file)
    
    with open('Prospect_framework_q_value_beta_test_'+str(beta_data)+'.pickle', 'wb') as file:
        pickle.dump(q_values_across_seeds, file)
    
